In [1]:
import awswrangler as wr

import mlflow

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


# Entrenamos el modelo con Decision Tree Classifier

Probaremos el modelo con DTC, con búsqueda de hiperparámetros mediante optuna.

In [2]:
mlflow_server = "http://localhost:5001"
mlflow.set_tracking_uri(mlflow_server)
s3_base_path = "s3://data/chicago/crimes/2024"

In [3]:
# Cargamos los datos para realizar nuestro estudio.
X_train =  wr.s3.read_csv(f"{s3_base_path}/final/X_train.csv")
y_train =  wr.s3.read_csv(f"{s3_base_path}/final/y_train.csv")

X_test =  wr.s3.read_csv(f"{s3_base_path}/final/X_test.csv")
y_test =  wr.s3.read_csv(f"{s3_base_path}/final/y_test.csv")

## Arrancamos a experimentar

In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Creamos una función para mostrar métricas
def metrics(model, X_test, y_test):

    y_pred = model.predict(X_test)
    cr = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    # Graficamos matriz de confusion
    categories = y_test["fbi_code"].astype("category").cat.categories
    cm = confusion_matrix(y_test, y_pred, labels=categories)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories)
    fig, ax = plt.subplots(figsize=(15,15))
    ax.grid(False)
    disp.plot(ax=ax)
    plt.close(fig)

    return cr, fig

Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [5]:
import datetime
from mlflow_aux import get_or_create_experiment
# Creemos el experimento
experiment_id = get_or_create_experiment("Chicago Crimes 2024")
print(experiment_id)
run_name_parent = "dtc_test"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


In [ ]:
from mlflow.models import infer_signature
import optuna 
from optuna_aux import champion_callback, objective
from sklearn.tree import DecisionTreeClassifier

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Creamos un estudio de Optuna
    # Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
    optuna.logging.set_verbosity(optuna.logging.ERROR)

    study = optuna.create_study(direction="maximize")
    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    study.optimize(lambda trial: objective(trial, X_train, y_train["fbi_code"], experiment_id), n_trials=30, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1_weighted", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Chicago Crimes 2024",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Creamos el arbol con los mejores parámetros obtenidos
    dec_tree_class_best = DecisionTreeClassifier(**study.best_params, random_state=42)
    # Entrenamos
    dec_tree_class_best.fit(X_train, y_train["fbi_code"])

    # Testeamos el modelo y logueamos el resultado
    cr, cm = metrics(dec_tree_class_best, X_test, y_test)

    # Logging all metrics in classification_report
    mlflow.log_metric("test_accuracy", cr.pop("accuracy"))
    for class_or_avg, metrics_dict in cr.items():
        for metric, value in metrics_dict.items():
            mlflow.log_metric("test_" + class_or_avg + '_' + metric,value)
    mlflow.log_figure(figure=cm, artifact_file="fda_cm.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, dec_tree_class_best.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=dec_tree_class_best,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="dtc_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)


c:\Users\Usuario\Documents\Filen.IA\Bimestre_3\MLOps1\aprendizaje_maquina_II\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


🏃 View run Trial: 0 at: http://localhost:5001/#/experiments/1/runs/0bb5d6cb49b14464ad9a35cf331576ab
🧪 View experiment at: http://localhost:5001/#/experiments/1
Initial trial 0 achieved value: 0.9775420295938944


c:\Users\Usuario\Documents\Filen.IA\Bimestre_3\MLOps1\aprendizaje_maquina_II\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


🏃 View run Trial: 1 at: http://localhost:5001/#/experiments/1/runs/63e35cc8633d4a498bb41fee58f42e20
🧪 View experiment at: http://localhost:5001/#/experiments/1
Trial 1 achieved value: 0.9988674402454938 with  2.1350% improvement


c:\Users\Usuario\Documents\Filen.IA\Bimestre_3\MLOps1\aprendizaje_maquina_II\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

🏃 View run Trial: 28 at: http://localhost:5001/#/experiments/1/runs/b6115fe14de94e868ea20a8d2f52165b
🧪 View experiment at: http://localhost:5001/#/experiments/1
Trial 28 achieved value: 0.9996509532077287 with  0.0000% improvement


c:\Users\Usuario\Documents\Filen.IA\Bimestre_3\MLOps1\aprendizaje_maquina_II\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


🏃 View run Trial: 29 at: http://localhost:5001/#/experiments/1/runs/25d39419b59d4ef8be0893438ebf31a6
🧪 View experiment at: http://localhost:5001/#/experiments/1
Trial 29 achieved value: 0.9997541608012339 with  0.0103% improvement


2025/08/05 02:29:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'dtc_model_dev'.
2025/08/05 02:29:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: dtc_model_dev, version 1
Created version '1' of model 'dtc_model_dev'.


🏃 View run dtc_test2025/08/05-02:28:14" at: http://localhost:5001/#/experiments/1/runs/82a240de0f2b4051b55242aeb01e4214
🧪 View experiment at: http://localhost:5001/#/experiments/1


## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
name = "dtc_model_prod"
desc = "This classifier show FBI code based on Chicago Police Report"

# Creamos el modelo productivo, si existe, lo actualizamos.
try:
    client.create_registered_model(name=name, description=desc)
except:
    client.update_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = dec_tree_class_best.get_params()
tags["model"] = type(dec_tree_class_best).__name__
tags["f1-score"] = cr["weighted avg"]["f1-score"]

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2025/08/05 02:29:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: dtc_model_prod, version 1
